# Importing necessary libraries 

In [1]:
import json
import torch
import string
import pandas as pd
from d2l import torch as d2l
from tf_keras.preprocessing.text import Tokenizer

2024-03-26 00:57:24.107056: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Data preprocessing

## 1. Reading the json file

In [2]:
#Reading the json file
file_path = 'train.json.zip'
df = pd.read_json(file_path)
df

,document,full_text,tokens,trailing_whitespace,labels
0,7,Design Thinking for innovation reflexion-Avril...,"[Design, Thinking, for, innovation, reflexion,...","[True, True, True, True, False, False, True, F...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,Diego Estrada\n\nDesign Thinking Assignment\n\...,"[Diego, Estrada, \n\n, Design, Thinking, Assig...","[True, False, False, True, True, False, False,...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,Reporting process\n\nby Gilberto Gamboa\n\nCha...,"[Reporting, process, \n\n, by, Gilberto, Gambo...","[True, False, False, True, True, False, False,...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,Design Thinking for Innovation\n\nSindy Samaca...,"[Design, Thinking, for, Innovation, \n\n, Sind...","[True, True, True, False, False, True, False, ...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,Assignment: Visualization Reflection Submitt...,"[Assignment, :, , Visualization, , Reflecti...","[False, False, False, False, False, False, Fal...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...,...,...
6802,22678,EXAMPLE – JOURNEY MAP\n\nTHE CHALLENGE My w...,"[EXAMPLE, –, JOURNEY, MAP, \n\n, THE, CHALLENG...","[True, True, True, False, False, True, True, F...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6803,22679,Why Mind Mapping?\n\nMind maps are graphical r...,"[Why, Mind, Mapping, ?, \n\n, Mind, maps, are,...","[True, True, False, False, False, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6804,22681,"Challenge\n\nSo, a few months back, I had chos...","[Challenge, \n\n, So, ,, a, few, months, back,...","[False, False, False, True, True, True, True, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6805,22684,Brainstorming\n\nChallenge & Selection\n\nBrai...,"[Brainstorming, \n\n, Challenge, &, Selection,...","[False, False, True, True, False, False, True,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


In [3]:
# get relevant columns
# clean full_text and remove extra characters 
# create word index
# encode bio labels
# explode data to get tokens

## 2. Extracting relevant columns

In [4]:
data = df[['document', 'tokens', 'labels']]
data['tokens'][0]
len(data['tokens'][0])

753

## 3. Cleaning tokens columns in data set

In [5]:
def preprocess(tokens):
    lowercase_tokens = [token.lower() for token in tokens]
    return lowercase_tokens

In [6]:
# def preprocess(tokens):
#     tokens = ' '.join(tokens)
#     # tokens_no_punct = tokens.translate(str.maketrans('', '', string.punctuation))
#     tokens_lower = tokens.lower()
#     processed_tokens = tokens_lower.split()
#     return processed_tokens

In [7]:
data['processed_tokens'] = data['tokens'].apply(preprocess)
data = data[['document', 'processed_tokens', 'labels']]
data

/var/folders/h3/ngfp429s6d3d3kvbxfb1q9pm0000gn/T/ipykernel_62658/1502291209.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['processed_tokens'] = data['tokens'].apply(preprocess)


,document,processed_tokens,labels
0,7,"[design, thinking, for, innovation, reflexion,...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,"[diego, estrada, \n\n, design, thinking, assig...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,"[reporting, process, \n\n, by, gilberto, gambo...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,"[design, thinking, for, innovation, \n\n, sind...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,"[assignment, :, , visualization, , reflecti...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...
6802,22678,"[example, –, journey, map, \n\n, the, challeng...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6803,22679,"[why, mind, mapping, ?, \n\n, mind, maps, are,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6804,22681,"[challenge, \n\n, so, ,, a, few, months, back,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6805,22684,"[brainstorming, \n\n, challenge, &, selection,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


## 4. Creating word index for processed_tokens examples in data

In [8]:
tokens = data['processed_tokens']

In [9]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokens)
word_index = tokenizer.word_index
word_index

{'the': 1,
 ' ': 2,
 '.': 3,
 'to': 4,
 ',': 5,
 'and': 6,
 'of': 7,
 '\n\n': 8,
 'a': 9,
 'in': 10,
 'i': 11,
 'we': 12,
 'that': 13,
 'for': 14,
 'is': 15,
 'with': 16,
 'it': 17,
 'this': 18,
 'was': 19,
 'as': 20,
 'our': 21,
 'on': 22,
 'be': 23,
 'their': 24,
 '\t\r \xa0': 25,
 '-': 26,
 'they': 27,
 'have': 28,
 'my': 29,
 'tool': 30,
 'are': 31,
 'from': 32,
 'what': 33,
 'were': 34,
 'them': 35,
 'not': 36,
 'by': 37,
 'or': 38,
 'an': 39,
 'can': 40,
 ':': 41,
 'more': 42,
 '\xa0': 43,
 'team': 44,
 'design': 45,
 'all': 46,
 ')': 47,
 'would': 48,
 'will': 49,
 'which': 50,
 'challenge': 51,
 'process': 52,
 'how': 53,
 'also': 54,
 'about': 55,
 'time': 56,
 'mind': 57,
 '(': 58,
 'had': 59,
 'people': 60,
 'learning': 61,
 'at': 62,
 'approach': 63,
 'one': 64,
 'use': 65,
 'new': 66,
 'so': 67,
 'thinking': 68,
 'but': 69,
 'different': 70,
 'visualization': 71,
 'you': 72,
 'work': 73,
 'us': 74,
 'launch': 75,
 'problem': 76,
 'ideas': 77,
 'could': 78,
 'each': 79,
 'a

In [10]:
len(word_index)

44390

## 5. Placing encoded tokens in data

In [11]:
encoded_tokens = tokenizer.texts_to_sequences(tokens)

In [12]:
data['encoded_tokens'] = encoded_tokens
data = data[['document', 'processed_tokens', 'encoded_tokens', 'labels']]
data

,document,processed_tokens,encoded_tokens,labels
0,7,"[design, thinking, for, innovation, reflexion,...","[45, 68, 14, 271, 8702, 26, 16418, 3564, 26, 1...","[O, O, O, O, O, O, O, O, O, B-NAME_STUDENT, I-..."
1,10,"[diego, estrada, \n\n, design, thinking, assig...","[11538, 16424, 8, 45, 68, 624, 8, 71, 30, 8, 5...","[B-NAME_STUDENT, I-NAME_STUDENT, O, O, O, O, O..."
2,16,"[reporting, process, \n\n, by, gilberto, gambo...","[1723, 52, 8, 37, 24720, 24721, 8, 51, 8, 11, ...","[O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT, O..."
3,20,"[design, thinking, for, innovation, \n\n, sind...","[45, 68, 14, 271, 8, 24722, 24723, 8, 19273, 6...","[O, O, O, O, O, B-NAME_STUDENT, I-NAME_STUDENT..."
4,56,"[assignment, :, , visualization, , reflecti...","[624, 41, 251, 71, 43, 205, 251, 2360, 43, 37,...","[O, O, O, O, O, O, O, O, O, O, O, O, B-NAME_ST..."
...,...,...,...,...
6802,22678,"[example, –, journey, map, \n\n, the, challeng...","[257, 126, 304, 188, 8, 1, 51, 109, 29, 3938, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6803,22679,"[why, mind, mapping, ?, \n\n, mind, maps, are,...","[247, 57, 84, 135, 8, 57, 515, 31, 1875, 3198,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6804,22681,"[challenge, \n\n, so, ,, a, few, months, back,...","[51, 8, 67, 5, 9, 311, 595, 354, 5, 11, 59, 99...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
6805,22684,"[brainstorming, \n\n, challenge, &, selection,...","[510, 8, 51, 134, 115, 8, 510, 15, 9, 113, 678...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."


## 6. Exploding tokens and labels

In [13]:
len(data['encoded_tokens'][0])

753

In [14]:
len(data['labels'][0])

753

In [15]:
data = data.explode(['encoded_tokens', 'labels'])
data

,document,processed_tokens,encoded_tokens,labels
0,7,"[design, thinking, for, innovation, reflexion,...",45,O
0,7,"[design, thinking, for, innovation, reflexion,...",68,O
0,7,"[design, thinking, for, innovation, reflexion,...",14,O
0,7,"[design, thinking, for, innovation, reflexion,...",271,O
0,7,"[design, thinking, for, innovation, reflexion,...",8702,O
...,...,...,...,...
6806,22687,"[mind, mapping, \n\n, challenge, \n\n, my, con...",52,O
6806,22687,"[mind, mapping, \n\n, challenge, \n\n, my, con...",836,O
6806,22687,"[mind, mapping, \n\n, challenge, \n\n, my, con...",642,O
6806,22687,"[mind, mapping, \n\n, challenge, \n\n, my, con...",3,O


## 7. Splitting data into training and test sets

In [16]:
num_rows_test = int(0.1 * len(data))
test_set = data.head(num_rows_test)
train_set = data.iloc[num_rows_test:]

In [17]:
train_set

,document,processed_tokens,encoded_tokens,labels
614,8854,"[reflection, –, storytelling, \n\n, challenge,...",1,O
614,8854,"[reflection, –, storytelling, \n\n, challenge,...",574,O
614,8854,"[reflection, –, storytelling, \n\n, challenge,...",17222,O
614,8854,"[reflection, –, storytelling, \n\n, challenge,...",78,O
614,8854,"[reflection, –, storytelling, \n\n, challenge,...",154,O
...,...,...,...,...
6806,22687,"[mind, mapping, \n\n, challenge, \n\n, my, con...",52,O
6806,22687,"[mind, mapping, \n\n, challenge, \n\n, my, con...",836,O
6806,22687,"[mind, mapping, \n\n, challenge, \n\n, my, con...",642,O
6806,22687,"[mind, mapping, \n\n, challenge, \n\n, my, con...",3,O


In [18]:
test_set

,document,processed_tokens,encoded_tokens,labels
0,7,"[design, thinking, for, innovation, reflexion,...",45,O
0,7,"[design, thinking, for, innovation, reflexion,...",68,O
0,7,"[design, thinking, for, innovation, reflexion,...",14,O
0,7,"[design, thinking, for, innovation, reflexion,...",271,O
0,7,"[design, thinking, for, innovation, reflexion,...",8702,O
...,...,...,...,...
614,8854,"[reflection, –, storytelling, \n\n, challenge,...",14,O
614,8854,"[reflection, –, storytelling, \n\n, challenge,...",1,O
614,8854,"[reflection, –, storytelling, \n\n, challenge,...",574,O
614,8854,"[reflection, –, storytelling, \n\n, challenge,...",10611,O


## 6. Encoding BIO format labels

In [20]:
encoded_labels = pd.get_dummies(data['labels'])
encoded_labels

,B-EMAIL,B-ID_NUM,B-NAME_STUDENT,B-PHONE_NUM,B-STREET_ADDRESS,B-URL_PERSONAL,B-USERNAME,I-ID_NUM,I-NAME_STUDENT,I-PHONE_NUM,I-STREET_ADDRESS,I-URL_PERSONAL,O
0,False,False,False,False,False,False,False,False,False,False,False,False,True
0,False,False,False,False,False,False,False,False,False,False,False,False,True
0,False,False,False,False,False,False,False,False,False,False,False,False,True
0,False,False,False,False,False,False,False,False,False,False,False,False,True
0,False,False,False,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6806,False,False,False,False,False,False,False,False,False,False,False,False,True
6806,False,False,False,False,False,False,False,False,False,False,False,False,True
6806,False,False,False,False,False,False,False,False,False,False,False,False,True
6806,False,False,False,False,False,False,False,False,False,False,False,False,True
